In [2]:
from database_cloudant import CloudantDB
from cloudant.design_document import DesignDocument
from cloudant.view import View
import sys



#db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
#db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
#db3 = CloudantDB("tweets_raw")         # raw tweet data in the specified locations
#db4=CloddantDB("")
#print(len(db1.get_data()))
#print(db2.get_data(n=10000))
#print(len(db3.get_data(n=10)))



    
def dataofday(date):    
    # get all data from a specific day
    db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
    db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
    n = 100  # number of tweets need to retrieve
    ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
    ddoc.fetch()
    view = View(ddoc,"view_time")
    data = []
    for row in view(group=False,limit=n,reduce=False,key=date,include_docs=True)["rows"]:
        data.append(row)
    
    db1.close()
    db2.close()
    return data

    
    

    
#db3.close()
def city_daily_popular_10words(tweetlist):
    city=[]
    for k in tweetlist:
        if k["zone"] not in city:
            city.append(k["zone"])
    alltext={}
    for c in city:
        alltext[c]=[]
    for k in tweetlist:
        for c in city:
            if k["zone"] in c:
                alltext[c].append(k["text"])
    popdict={}             
    for c in city:
        popdict[c]=top_words(alltext[c])
    return popdict


if __name__ == "main":
    ipaddress=sys.argv[-1]

    date = "2021-01-01"
    data=dataofday(date)

    citytopwords=city_daily_popular_10words(data)
    print(citytopwords)


In [60]:
def get_data_summary(db="covid",viewType="month",startkey="2020-10",mode = "mean"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return {}
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="month"):
        view = View(ddoc, 'view_month')
    elif(viewType=="zone"):
        view = View(ddoc,"view_zone")
    elif(viewType=="zone month"):
        view=View(ddoc, 'view_zone_month')
    else:
        view = View(ddoc, 'view_time')
    result = {}
    for row in view(limit=1000000,reduce=True,group=True,startkey=startkey)['rows']:
        if(mode=="mean"):
            result[row['key']] = round(row['value']['sum']/row['value']['count'],6)
        elif(mode=="count"):
            result[row['key']] = row['value']['count']
    return result

def get_data(n=100,db="covid",viewType="day",startkey="2021-05"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return []
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="zone"):
        view = View(ddoc,"view_zone")
    elif(viewType=="day"):
        view = View(ddoc, 'view_time')
    elif(viewType=="month"):
        view = View(ddoc, 'view_month')
    else:
        return "Does not support viewType"+viewType
    result = []
    for row in view(limit=n,reduce=False,group=False,startkey=startkey,include_docs=True)['rows']:
        result.append(row)
    return result


In [63]:
get_data(db="covid",viewType="month")

Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid


[{'id': '1388298424122888192',
  'key': '2021-05',
  'value': 0.109441996,
  'doc': {'_id': '1388298424122888192',
   '_rev': '1-9d51086f10e8ffc7579e24203d7dc680',
   'id': 1388298424122888192,
   'user_id': 822557964,
   'created_at': '2021-05-01 01:05:12',
   'text': 'Complete lockdown is the only medicine at this time .Stop more misery for the people . Night curfew is not any curf… https://t.co/fUIpsP7EHW',
   'place': {'type': 'city',
    'name': 'Sydney',
    'fullname': 'Sydney, New South Wales',
    'bounding_box': '[[[150.520928608, -34.1183470085], [151.343020992, -34.1183470085], [151.343020992, -33.578140996], [150.520928608, -33.578140996]]]'},
   'sentiment_score': '0.109441996',
   'sentiment': 'Negative',
   'zone': 'Sydney',
   'keywords': 'lockdown'}},
 {'id': '1388321095179141121',
  'key': '2021-05',
  'value': 0.46379364,
  'doc': {'_id': '1388321095179141121',
   '_rev': '1-881755ab0cd39740049641bd4f658a27',
   'id': 1388321095179141121,
   'user_id': 206603052,
  

In [5]:
monthsentidict=get_data_summary(viewType="zone")

Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid


In [6]:
get_data_summary(viewType="zone month")
get_data_summary(db="vaccine",viewType="zone month")
get_data_summary(db="vaccine",viewType="zone")
get_data_summary(viewType="zone")

Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid
Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_vaccine
Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_vaccine
Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid


{'Adelaide': 0.596067,
 'Ballarat': 0.737719,
 'Brisbane': 0.567257,
 'Bunbury': 0.688183,
 'Canberra': 0.518975,
 'Geelong': 0.611571,
 'Hobart': 0.48139,
 'Melbourne': 0.589867,
 'Newcastle': 0.714606,
 'Perth': 0.56422,
 'Sydney': 0.542216}

In [7]:
monthsentidict

{'Adelaide': 0.596067,
 'Ballarat': 0.737719,
 'Brisbane': 0.567257,
 'Bunbury': 0.688183,
 'Canberra': 0.518975,
 'Geelong': 0.611571,
 'Hobart': 0.48139,
 'Melbourne': 0.589867,
 'Newcastle': 0.714606,
 'Perth': 0.56422,
 'Sydney': 0.542216}

In [8]:
get_data(db="no_keywords",viewType="month")

Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords


'Does not support viewTypemonth'

In [54]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords


def top_words(textlist,top=10):
    stop_words = stopwords.words('english')
    wordfreqdict={}
    popular=[]
    tokenizer=TweetTokenizer()
    for t in textlist:
        token = tokenizer.tokenize(t)
        for w in token:
            if w not in wordfreqdict.keys():
                wordfreqdict[w]=1
            else:
                wordfreqdict[w]+=1
    sorted_values = sorted(wordfreqdict.values(), reverse=True) # Sort the values
    count=0
    for i in sorted_values:
        for k in wordfreqdict.keys():
            if wordfreqdict[k] == i:
                if len(k)>=5 and k not in popular and k not in stop_words:
                    
                    popular.append(k)
                    count+=1
                if count > top:
                    break
             
    popdict={}
    for p in popular:
        popdict[p]=wordfreqdict[p]
    return popdict
def city_daily_popular_10words():

    city=[]
    tweetlist=get_data(n=3000,db="no_keywords",viewType="day",startkey="2021-05-13")
    for k in  tweetlist:
        if k["doc"]["zone"] not in city:
            city.append(k["doc"]["zone"])
    alltext={}
    for c in city:
        alltext[c]=[]
    for k in tweetlist:
        for c in city:
            if k["doc"]["zone"] in c:
                alltext[c].append(k["doc"]["text"])
    popdict={}             
    for c in city:
        popdict[c]=top_words(alltext[c])
    return popdict

In [55]:
cdict=city_daily_popular_10words()

Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords


In [56]:
cdict

{'Canberra': {'Morrison': 5,
  'gahod': 4,
  'Canberra': 4,
  'Labor': 4,
  'great': 4,
  'Friday': 4,
  '#NRLTigersKnights': 4,
  'Scott': 3,
  'government': 3,
  'women': 3,
  'better': 3,
  'mistakes': 2,
  'Important': 1},
 'Sydney': {'posted': 75,
  'Sydney': 70,
  'photo': 68,
  'Australia': 64,
  'today': 18,
  'people': 18,
  'really': 14,
  'years': 13,
  'going': 12,
  'think': 12,
  'great': 12,
  'never': 7,
  'everyone': 5,
  'games': 2,
  'https://t.co/BhmZhEIwuU': 1},
 'Brisbane': {'posted': 23,
  'photo': 20,
  'Brisbane': 13,
  'Queensland': 10,
  'Australia': 10,
  'Rainbow': 8,
  'Crystal': 8,
  'Magik': 8,
  'people': 6,
  'first': 6,
  '#NRLMagicRound': 6,
  'today': 5,
  'think': 4},
 'Melbourne': {'posted': 67,
  'photo': 66,
  'Melbourne': 39,
  'Victoria': 35,
  'Australia': 27,
  'today': 19,
  'great': 17,
  'night': 17,
  'Drinking': 17,
  'South': 13,
  'Eastern': 13,
  'video': 6,
  'young': 5,
  'Mubarak': 4,
  '@venus_babes': 3,
  'beautiful': 2,
  'sadd

In [34]:
get_data(db="no_keywords",viewType="day")

Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords


[{'id': '1388282235938394130',
  'key': '2021-05-01',
  'value': 0.98362815,
  'doc': {'_id': '1388282235938394130',
   '_rev': '1-bdd52374143a46cb27c3e48f87628d8e',
   'id': 1388282235938394130,
   'user_id': 2225880756,
   'created_at': '2021-05-01 00:00:52',
   'text': 'Just posted a photo @ Bondi Beach https://t.co/wL81mg2GsI',
   'place': {'type': 'city',
    'name': 'Sydney',
    'fullname': 'Sydney, New South Wales',
    'bounding_box': '[[[150.520928608, -34.1183470085], [151.343020992, -34.1183470085], [151.343020992, -33.578140996], [150.520928608, -33.578140996]]]'},
   'sentiment_score': '0.98362815',
   'sentiment': 'Positive',
   'zone': 'Sydney',
   'keywords': 'no keywords'}},
 {'id': '1388282374153117700',
  'key': '2021-05-01',
  'value': 0.76601136,
  'doc': {'_id': '1388282374153117700',
   '_rev': '1-02412e724ab78d53e8263167a034226e',
   'id': 1388282374153117700,
   'user_id': 835298557,
   'created_at': '2021-05-01 00:01:25',
   'text': 'Professor Clifton provide

In [39]:
cdict.keys()

dict_keys(['Sydney', 'Adelaide', 'Melbourne', 'Newcastle', 'Brisbane', 'Canberra', 'Perth', 'Ballarat', 'Hobart', 'Geelong'])

In [36]:
#from database_cloudant import CloudantDB
#from cloudant.design_document import DesignDocument
db10 = CloudantDB("topwords")
db10.add_record({"word1":cdict,"word2":"value2","area":[1,2,3]},key="2021-05-01")
db10.add_record({"word1":123,"area":[1,2,3]},key="2020-02-02")

print(db10.curDB["2020-02-03"])
print()
print(db10.get_data())


Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: topwords
{'_id': '2020-02-03', '_rev': '1-08c8236d7ce24dd92436942e927e1afc', 'aaa': 'ddd'}

[{'_id': '2020-02-02', '_rev': '1-c4b93165638b03835d4bd41edb8e46c4', 'word1': 123, 'area': [1, 2, 3]}, {'_id': '2020-02-03', '_rev': '1-08c8236d7ce24dd92436942e927e1afc', 'aaa': 'ddd'}, {'_id': '2021-05-01', '_rev': '1-b7d91b5553faecd437f55c4645c1cf59', 'word1': {'Sydney': {'posted': 108, 'photo': 103, 'Australia': 80, 'Sydney': 76, 'today': 25, 'first': 18, 'video': 14, 'South': 14, 'Monday': 13, 'morning': 12, 'really': 12, 'people': 9, 'Saturday': 8, 'Bondi': 7, 'Beach': 5, 'Point': 4, 'https://t.co/wL81mg2GsI': 1}, 'Adelaide': {'Adelaide': 12, 'Australian': 8, "Women's": 8, 'Baseball': 8, 'Showcase': 8, 'today': 8, '@baseballcomau': 7, 'Australia': 6, 'South': 5, 'posted': 5, 'photo': 5, '#morninginspiration': 4, 'Professor': 1}, 'Melbourne': 

In [22]:
# -*- coding: utf-8 -*-
"""
Created on Wed May 19 00:40:53 2021

@author: Windwalker
"""
USERNAME='admin'
PASSWORD = 'password'
URL = 'http://172.26.131.97:5984'
from cloudant.client import CouchDB
from cloudant.design_document import DesignDocument
from cloudant.view import View

class CloudantDB():
    def __init__(self,db_name,username = USERNAME,password=PASSWORD,url = URL,partition=True):
        self.client = CouchDB(USERNAME, PASSWORD, url=URL, connect=True)
        self.session = self.client.session()
        self.curDB = None
        print('Username: {0}'.format(self.session['userCtx']['name']))
        print('Databases: {0}'.format(self.client.all_dbs()))
        self.accessDB(db_name,partition=True)
    
    def accessDB(self,db_name,partition=True):
        if(db_name in self.client.all_dbs()):
            self.curDB = self.client[db_name]
        else:
            self.curDB = self.client.create_database(db_name)
            print("creating db:",db_name)
        if self.curDB.exists():
            print('Accessing db:',db_name)
    
    def add_record(self, json_record, db_name=None, key=None):
        if(key is not None and key not in self.curDB):
            json_record['_id'] = key
            self.curDB.create_document(json_record)
        elif('id' in json_record.keys()):
            hash_string = str(hash(json_record['id']))
            if(hash_string not in self.curDB):
                json_record['_id'] = hash_string
                record = json_record
                self.curDB.create_document(record)
            else:
                print("already exist")
    
    def delete_record(self,recordID):
        if(recordID in self.curDB):
            my_document = self.curDB[recordID]
            my_document.delete()
        elif(str(hash(recordID)) in self.curDB):
            my_document = self.curDB[str(hash(recordID))]
            my_document.delete()
        else:
            print("deletion err, record not found")
    def delete_all_records(self):
        for doc in self.curDB:
            doc.delete()
    
    def deleteDB(self,db_name):
        self.client.delete_database(db_name)
        self.curDB = None
        print(db_name,"removed")
    
    def get_data(self,n=10000):
        data = []
        count = 0
        for document in self.curDB:
            data.append(document)
            count+=1
            if(count>n):
                break
        return data
    
    def close(self):
        self.client.disconnect()
        
    def allDB(self):
        all_dbs = self.client.all_dbs()
        print('Databases: {0}'.format(all_dbs))
        return all_dbs
"""
reduce_func = "function(key, values, rereduce) {var result = 0;for(var i = 0; i < values.length; i++){value = parseFloat(values[i]);result+=value;}return result.toString();}"
db2 = CloudantDB("tweets_no_keywords")

ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
ddoc.fetch()
ddoc.add_view("view_zone",'function(doc){emit(doc.zone,parseFloat(doc.sentiment_score))}',"_stats")
ddoc.save()

view = View(ddoc, 'view_zone')
# Assuming that 'view001' exists as part of the
# design document ddoc in the remote database...
# Use view as a callable
for row in view(limit=100,group=True)['rows']:
    print(row)
for view_name, view in ddoc.iterviews():
    print("------",view_name,view)

db2 = CloudantDB("tweets_no_keywords")
ddoc = DesignDocument(db2.curDB,'_design/ddoc001')
ddoc.fetch()
ddoc.add_view("view_time","function(doc){emit(doc.created_at.split(' ')[0], parseFloat(doc.sentiment_score))}","_stats")
ddoc.save()
view = View(ddoc, 'view_time')
for row in view(limit=100,reduce=True,group=True)['rows']:
    print(row)
"""

'\nreduce_func = "function(key, values, rereduce) {var result = 0;for(var i = 0; i < values.length; i++){value = parseFloat(values[i]);result+=value;}return result.toString();}"\ndb2 = CloudantDB("tweets_no_keywords")\n\nddoc = DesignDocument(db2.curDB, \'_design/ddoc001\')\nddoc.fetch()\nddoc.add_view("view_zone",\'function(doc){emit(doc.zone,parseFloat(doc.sentiment_score))}\',"_stats")\nddoc.save()\n\nview = View(ddoc, \'view_zone\')\n# Assuming that \'view001\' exists as part of the\n# design document ddoc in the remote database...\n# Use view as a callable\nfor row in view(limit=100,group=True)[\'rows\']:\n    print(row)\nfor view_name, view in ddoc.iterviews():\n    print("------",view_name,view)\n\ndb2 = CloudantDB("tweets_no_keywords")\nddoc = DesignDocument(db2.curDB,\'_design/ddoc001\')\nddoc.fetch()\nddoc.add_view("view_time","function(doc){emit(doc.created_at.split(\' \')[0], parseFloat(doc.sentiment_score))}","_stats")\nddoc.save()\nview = View(ddoc, \'view_time\')\nfor ro